In [1]:
import numpy as np
import random

COORDS_1 = [-1, 1]
COORDS_2 = [1, 0, -1]
COORDS_3 = [np.array([-x, y, z]) for z in COORDS_2 for y in COORDS_2 for x in COORDS_2]

NORMS = {"U": np.array([0, 0, 1]), "D": np.array([0, 0, -1]), "F": np.array([0, -1, 0]),
         "B": np.array([0, 1, 0]), "R": np.array([1, 0, 0]), "L": np.array([-1, 0, 0]), }

C_WISE = {"U": np.array([[0, -1, 0], [1, 0, 0], [0, 0, 1]]), "F": np.array([[0, 0, -1], [0, 1, 0], [1, 0, 0]]),
          "R": np.array([[1, 0, 0], [0, 0, -1], [0, 1, 0]]), "D": np.array([[0, 1, 0], [-1, 0, 0], [0, 0, 1]]),
          "B": np.array([[0, 0, 1], [0, 1, 0], [-1, 0, 0]]), "L": np.array([[1, 0, 0], [0, 0, 1], [0, -1, 0]])}
OPPS = {"U": "D", "D": "U", "R": "L", "L": "R", "F": "B", "B": "F"}

SIDES = ["U", "F", "R", "B", "L", "D"]
SIDES2 = ["Up", "Front", "Right", "Back", "Left", "Down"]

COLOURS = ["W", "G", "R", "B", "O", "Y"]

class Face:
    def __init__(self, norm, colour="Black"):
        self.norm = norm
        self.colour = colour
        # self.colour

    def copy(self):
        """Creates a deep copy of a Face."""
        news = Face(np.array(self.norm[:]), self.colour[:])
        return news

    def __str__(self):
        return "Normal:" + str(self.norm) + "\nColour:" + self.colour


class Cubie:
    def __init__(self, coordinates, faces=[]):
        self.coordinates = coordinates
        self.faces = faces

        # Initialising 6 faces per Cubie with norms in x, y, and z directions
        if self.faces == []:
            for i in range(3):
                for j in COORDS_1:
                    norm = np.zeros(3)
                    norm[i] = j
                    self.faces.append(Face(norm))

    def copy(self):
        """Creates a deep copy of a Cubie. For use by operators in creating new states."""
        news = Cubie(self.coordinates[:])
        news.faces = [face.copy() for face in self.faces]
        return news

    def x(self):
        return self.coordinates[0]

    def y(self):
        return self.coordinates[1]

    def z(self):
        return self.coordinates[2]

    def in_side(self, side):
        """Verifies if the Cubie is contained in the side given."""
        if side == "U":
            return self.z() == 1
        if side == "D":
            return self.z() == -1
        if side == "F":
            return self.y() == -1
        if side == "B":
            return self.y() == 1
        if side == "R":
            return self.x() == 1
        if side == "L":
            return self.x() == -1

# Generating a list of cubies with a solved arrangement
solved_cubies = [Cubie(np.array([x, y, z])) for x in COORDS_2 for y in COORDS_2 for z in COORDS_2]


class Cube:
    def __init__(self, cubies=solved_cubies):
        self.cubies = cubies

        # Initialising cubies with non-black colours on their outward faces
        if cubies == solved_cubies:
            for cubie in self.cubies:
                for face in cubie.faces:
                    if cubie.z() == 1 and face.norm[2] == 1:
                        face.colour = "W"
                    if cubie.z() == -1 and face.norm[2] == -1:
                        face.colour = "W"
                    if cubie.x() == 1 and face.norm[0] == 1:
                        face.colour = "R"
                    if cubie.x() == -1 and face.norm[0] == -1:
                        face.colour = "R"
                    if cubie.y() == 1 and face.norm[1] == 1:
                        face.colour = "G"
                    if cubie.y() == -1 and face.norm[1] == -1:
                        face.colour = "G"

    def copy(self):
        """Performs a deep copy of a Cube for use in operators in creating new states."""
        new_cubies = [cubie.copy() for cubie in self.cubies]
        news = Cube(new_cubies)
        return news

    def move(self, side, number_of_turns=1):
        """Given a side to turn and the number of time to turn it 90 degrees clockwise, returns a new cube having
        performed this action.
        """
        news = self.copy()
        turn = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

        # Creating a matrix that will perform the desired rotation
        for i in range(number_of_turns):
            turn = turn.dot(C_WISE[side])

        # Performing rotation on each Cubie's coordinates and the norms of its faces
        for i, cubie in enumerate(news.cubies):
            if self.cubies[i].in_side(side):
                for j in range(len(cubie.faces)):
                    old_face_norm = self.cubies[i].faces[j].norm
                    cubie.faces[j].norm = old_face_norm.dot(turn)
                cubie.coordinates = self.cubies[i].coordinates.dot(turn)
        return news

    def can_move(self, side, number_of_turns):
        """Required method for agent."""
        return True

    def cube_array(self):
        """Generates a 6 x 3 x 3 list representing the faces of the Rubiks cube."""
        cube_sides = {}

        for side in SIDES:
            cube_sides[side] = []

        for coord in COORDS_3:
            for cubie in self.cubies:
                if np.array_equal(cubie.coordinates, coord):  # Making sure that cubies are processed in correct order
                    for side in SIDES:
                        if cubie.in_side(side):
                            for face in cubie.faces:
                                if np.array_equal(face.norm, NORMS[side]):
                                    cube_sides[side].append(face.colour)

        new_list = [cube_sides["U"], cube_sides["F"], reversal(cube_sides["R"]), reversal(cube_sides["B"]),
                    cube_sides["L"], reversal(cube_sides["D"])]
        final_list = [nine_to_3x3(side) for side in new_list]
        return final_list

    def __str__(self):
        text = ""
        for i, side in enumerate(self.cube_array()):
            text += SIDES2[i] + " face\n"
            for line in side:
                text += str(line) + "\n"
        return text

    def __eq__(self, other):
        return self.cube_array() == other.cube_array()

#### HELPER FUNCTIONS

def reversal(side):
    """Takes a list of 9 like [1, 2, 3, 4, 5, 6, 7, 8, 9] and returns [3, 2, 1, 6, 5, 4, 9, 8, 7].
    Used for certain sides that are processed incorrectly"""
    new_side = []
    #     print(side)
    k = int(len(side) / 3)
    for i in range(k):
        for j in range(3):
            new_side.append(side[k * i - j + 2])
    return new_side


assert reversal([1, 2, 3, 4, 5, 6, 7, 8, 9]) == [3, 2, 1, 6, 5, 4, 9, 8, 7], "Reversal Fails"


def nine_to_3x3(listy):
    """Reshapes a list of length 9 to a 3 x 3 list."""
    new_side = []
    k = int(len(listy) / 3)
    for i in range(k):
        c = []
        for j in range(3):
            c.append(listy.pop(0))
        new_side.append(c)
    return new_side


assert nine_to_3x3([1, 2, 3, 4, 5, 6, 7, 8, 9]) == [[1, 2, 3], [4, 5, 6], [7, 8, 9]], "nine_to_3x3 Fails"

################

#### GOAL TESTING ####

GOAL_CUBE = Cube()


def goal_test(c):
    """Required method for solving agent"""
    return c == GOAL_CUBE


def goal_message(s):
    return "You've solved our rudimentary Rubik's Cube. Great!"


class Operator:
    def __init__(self, name, precond, state_transf):
        self.name = name
        self.precond = precond
        self.state_transf = state_transf

    def possible(self, s):
        return self.precond(s)

    def __call__(self, s):
        return self.state_transf(s)

    def __str__(self):
        return self.name


#### CREATING OPERATORS ####
OPERATORS = []
num_90 = [1, 3]  # Clockwise and Anticlockwise turns
# num_90 = [1, 2, 3]
for i in num_90:
    for side in SIDES:
        op = Operator("Turns the " + str(side) + " face clockwise of the cube " + str(i) + " times",
                      lambda c, side1=side, j=i: c.can_move(side1, j),
                      lambda c, side1=side, j=i: c.move(side1, j))
        OPERATORS.append(op)
#         print(op)

def scramble(cube, turn_number, show_turns=False):
    """Returns a scrambled Cube."""
    for i in range(turn_number):
        action = random.choice(OPERATORS)
        if show_turns:
            print()
            print(action)
        new_c = action(cube)
        cube = new_c
    return cube


#### INITIALISE A STARTING STATE ####

INITIAL_STATE = scramble(GOAL_CUBE, 30)

In [2]:
c = Cube()
print(c)

Up face
['W', 'W', 'W']
['W', 'W', 'W']
['W', 'W', 'W']
Front face
['G', 'G', 'G']
['G', 'G', 'G']
['G', 'G', 'G']
Right face
['R', 'R', 'R']
['R', 'R', 'R']
['R', 'R', 'R']
Back face
['G', 'G', 'G']
['G', 'G', 'G']
['G', 'G', 'G']
Left face
['R', 'R', 'R']
['R', 'R', 'R']
['R', 'R', 'R']
Down face
['W', 'W', 'W']
['W', 'W', 'W']
['W', 'W', 'W']



In [5]:
new_cube = scramble(c, 3, True)
print()
print(new_cube)


Turns the D face clockwise of the cube 3 times

Turns the F face clockwise of the cube 3 times

Turns the L face clockwise of the cube 3 times

Up face
['G', 'W', 'W']
['G', 'W', 'W']
['G', 'R', 'G']
Front face
['R', 'G', 'R']
['W', 'G', 'R']
['W', 'G', 'R']
Right face
['W', 'R', 'R']
['W', 'R', 'R']
['W', 'G', 'G']
Back face
['G', 'G', 'R']
['G', 'G', 'W']
['R', 'R', 'W']
Left face
['W', 'W', 'W']
['R', 'R', 'G']
['R', 'R', 'G']
Down face
['W', 'W', 'G']
['W', 'W', 'G']
['G', 'R', 'R']

